In [75]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [76]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from 'D:\\Anaconda\\Code\\EnSF\\AllenCahn\\AllenCahn_Solver.py'>

In [90]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
T = 10
Nt = 1000
eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

S = 2

cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [91]:
# scipy.io.savemat('RefU0_AllenC_v1.mat', {'U0': U0}) # save initial condition

In [83]:
data = scipy.io.loadmat('RefU0_AllenC_v1.mat')
# data = scipy.io.loadmat('RefU0_AllenC_ConsMob_v1.mat')
# Convert MATLAB arrays to NumPy arrays and cast to float64.
U0 = data['U0'].astype(np.float64)
U0 = U0.reshape(Nx*Ny, )

In [92]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

# Idd = np.eye(N)
# Dh = np.kron(Idd, G)+np.kron(G, Idd)

In [2]:
noise = 0*np.random.randn(Nx*Ny)
U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0, N, dt, eps, gamma, S, Dh, cons, noise)

In [94]:
RefSol = np.zeros((Nx*Ny, Nt+1))

RefSol[:, 0] += U0
RefSol[:, 1] += U1

In [1]:
for ii in range(1, Nt):
    print(f'step={ii}:')
    t1 = time.time() 
    x = AC_Neu.AllenCahn_Solver_1step_BDF2(U1, U0, N, dt, eps, gamma, S, Dh, b0, b1, cons)
    t2 = time.time()
    print(f'\t time = {t2 - t1:.4f}')
    
    U0 = U1.copy()
    U1 = x.copy()
    RefSol[:, ii+1] +=x

In [96]:
scipy.io.savemat('TestRefSol_AllenC_T10_ConsMob_v1.mat', {'RefU':RefSol})